In [31]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.metrics import accuracy_score

In [32]:
import os

liked_songs_file = 'liked_songs.csv'

# Check if the file exists
if os.path.exists(liked_songs_file):
    df_liked_songs = pd.read_csv(liked_songs_file)
else:
    df_liked_songs = pd.DataFrame()

In [33]:
df = pd.read_csv('../datasets/user_song_mockfeatures.csv')
df['likes_song'] = (df['affinity_score'] > df['affinity_score'].median()).astype(int)

In [34]:


label_encoder = LabelEncoder()
df['track_id_encoded'] = label_encoder.fit_transform(df['track_id'])


# Perform label encoding on the track_id column
df['track_id_encoded'] = label_encoder.fit_transform(df['track_id'])
# Move the 'track_id_encoded' column to the first position
df = df[['track_id_encoded'] + [col for col in df.columns if col != 'track_id_encoded']]
df[['user_id', 'gender', 'avg_bpm', 'age', 'avg_cadence'] + [col for col in df.columns if col not in ['user_id', 'gender', 'avg_bpm', 'age', 'avg_cadence']]]
df.drop(['user_id'], inplace=True, axis=1)
df.head()


,track_id_encoded,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,intensity_zone,intensity_zone_encoded,cluster,affinity_score,gender,avg_bpm,age,avg_cadence,avg_pace,likes_song
0,215,0hv6ku4431kPrjbpBBspw3,Animals - Original Mix,6098,5,0N5QcojgdA6aP9oQ7yYG9A,1148,2013-07-01,gym melbourne house,5BqOZpVEqRDfZScvW1QUyA,...,Zone 4 (Hard),3,10,0.287000,1,165.0,16.0,83.0,6.27,0
1,2080,6uYAPqxP69zfpCbsTHXgiz,Cater 2 U,2462,62,0b6ivSFfDs38MG7aLn9rvO,4221,2004-11-16,hip hop rnb,0Ar0Ng9DlAWZtSPBvOQgOa,...,Zone 3 (Moderate),2,14,0.504277,1,165.0,16.0,83.0,6.27,1
2,521,1gzIbdFnGJ226LTl0Cn2SX,White Wedding - Pt. 1,1083,68,5ebh0MxkqELmk50xHj2b4k,2079,1982-01-01,hard rock cafe,55ybnG2z8rpTIerRfnCsw6,...,Zone 5 (Maximum),4,0,0.395855,1,165.0,16.0,83.0,6.27,0
3,887,2vX03yHcVtFUg2ZMuzhLhS,Outlaw - Original Mix,5252,0,7jV5akfXEXsoNsqrekDCyX,12116,2014-04-21,latin village,3JcJCsvBnRsA6oc7eUYDSB,...,Zone 4 (Hard),3,10,0.285107,1,165.0,16.0,83.0,6.27,0
4,953,38LrhW3eQSt3cK88dkvl5e,Misty Night Cruising,5066,33,24sd1ZTrjjabj0A54XK0RP,11054,1984-12-21,city pop シティーポップ,3j2osvmecEao5nmo9jZ5df,...,Zone 5 (Maximum),4,0,0.292775,1,165.0,16.0,83.0,6.27,0


In [35]:
def df_preprocessing(df):
    # Create an instance of LabelEncoder

    model_df = df.copy()
    model_df = model_df.select_dtypes(exclude=['object'])
    model_df = df[['track_id'] + list(model_df.columns)] #creating a dataset with track_id + model_df columns

    return model_df
        

In [36]:
model_df = df_preprocessing(df)
model_df.drop(['track_album_name', 'intensity_zone_encoded', 'cluster', 'track_id'],inplace=True, axis=1)

/var/folders/gr/q7qsw30x1xqd0zv1tj58r0vw0000gn/T/ipykernel_68148/2579399489.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df.drop(['track_album_name', 'intensity_zone_encoded', 'cluster', 'track_id'],inplace=True, axis=1)


In [37]:
model_df.head()

,track_id_encoded,track_artist,track_popularity,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,speechiness,...,duration_minutes,playlist_name_sentiment,track_name_sentiment,affinity_score,gender,avg_bpm,age,avg_cadence,avg_pace,likes_song
0,215,6098,5,0,18,0.675,0.868,1,-6.360,0.0392,...,5.07,0.000000,0.375,0.287000,1,165.0,16.0,83.0,6.27,0
1,2080,2462,62,3,9,0.605,0.584,11,-7.043,0.1970,...,4.09,0.000000,0.000,0.504277,1,165.0,16.0,83.0,6.27,1
2,521,1083,68,5,7,0.673,0.814,9,-5.411,0.0367,...,4.20,-0.291667,0.000,0.395855,1,165.0,16.0,83.0,6.27,0
3,887,5252,0,1,11,0.557,0.979,6,-4.254,0.0489,...,4.81,0.000000,0.375,0.285107,1,165.0,16.0,83.0,6.27,0
4,953,5066,33,5,0,0.587,0.977,10,-3.589,0.0711,...,4.46,0.000000,0.000,0.292775,1,165.0,16.0,83.0,6.27,0


In [38]:
# Features and target variable
def dividing_df(df):
    X = model_df.drop(['likes_song', 'affinity_score'], axis=1)
    y = model_df['likes_song']
    categorical_features = ['user_id', 'gender', 'playlist_genre', 'playlist_subgenre', 'key', 'track_artist']
    numerical_features = [col for col in model_df.columns if col not in categorical_features and col not in ['likes_song', 'affinity_score'] and col in model_df.columns]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    return X_train, X_test, y_train, y_test
 


In [39]:
X_train, X_test, y_train, y_test = dividing_df(model_df)
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

In [40]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mse',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
}

In [41]:
gbm = lgb.train(params, train_data, num_boost_round=100, valid_sets=[test_data])

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000546 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3457
[LightGBM] [Info] Number of data points in the train set: 2000, number of used features: 23
[LightGBM] [Info] Start training from score 0.501000


In [42]:
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
y_pred_class = [1 if prob >= 0.5 else 0 for prob in y_pred]


In [43]:
X_test['y_pred'] = y_pred_class
X_test['y_test'] = y_test

In [44]:
mse = mean_squared_error(y_test, y_pred)
print(f"The Mean Squared Error (MSE) on test set: {mse}")

The Mean Squared Error (MSE) on test set: 0.14408894314795379


In [45]:
X_test.head()

,track_id_encoded,track_artist,track_popularity,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,speechiness,...,duration_minutes,playlist_name_sentiment,track_name_sentiment,gender,avg_bpm,age,avg_cadence,avg_pace,y_pred,y_test
1583,1314,5185,15,0,1,0.634,0.658,8,-10.435,0.0334,...,5.94,0.000000,0.0,0,104.0,26.0,32.0,7.99,1,0
1812,1101,5358,43,3,13,0.661,0.311,10,-7.848,0.0280,...,3.58,0.000000,0.0,1,81.0,57.0,95.0,7.08,0,0
1512,948,3896,25,5,2,0.516,0.720,6,-4.738,0.0356,...,3.66,0.166667,0.0,0,245.0,22.0,38.0,6.91,1,1
1878,1997,5454,84,2,17,0.703,0.594,5,-6.146,0.0752,...,1.91,0.000000,0.0,0,99.0,13.0,69.0,6.56,1,1
1777,344,7113,3,2,17,0.652,0.887,2,-4.002,0.2230,...,3.79,0.700000,0.0,0,65.0,39.0,90.0,6.02,1,0


In [46]:
from sklearn.metrics import accuracy_score
# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred_class)
print(f"The accuracy of the model is: {accuracy}")


The accuracy of the model is: 0.796


In [47]:
feature_importance = gbm.feature_importance()
feature_names = gbm.feature_name()

# Create a dictionary to store the feature importance
feature_importance_dict = dict(zip(feature_names, feature_importance))

# Sort the feature importance dictionary by value in descending order
sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

# Print the sorted feature importance
for feature, importance in sorted_feature_importance:
    print(f"{feature}: {importance}")


tempo: 378
loudness: 246
energy: 227
danceability: 184
acousticness: 169
avg_cadence: 155
speechiness: 153
liveness: 134
avg_bpm: 133
age: 133
track_id_encoded: 128
avg_pace: 128
valence: 125
duration_minutes: 119
track_popularity: 95
playlist_subgenre: 90
instrumentalness: 86
track_artist: 77
playlist_genre: 68
key: 60
playlist_name_sentiment: 53
track_name_sentiment: 32
gender: 27


In [48]:
model_df.head()


,track_id_encoded,track_artist,track_popularity,playlist_genre,playlist_subgenre,danceability,energy,key,loudness,speechiness,...,duration_minutes,playlist_name_sentiment,track_name_sentiment,affinity_score,gender,avg_bpm,age,avg_cadence,avg_pace,likes_song
0,215,6098,5,0,18,0.675,0.868,1,-6.360,0.0392,...,5.07,0.000000,0.375,0.287000,1,165.0,16.0,83.0,6.27,0
1,2080,2462,62,3,9,0.605,0.584,11,-7.043,0.1970,...,4.09,0.000000,0.000,0.504277,1,165.0,16.0,83.0,6.27,1
2,521,1083,68,5,7,0.673,0.814,9,-5.411,0.0367,...,4.20,-0.291667,0.000,0.395855,1,165.0,16.0,83.0,6.27,0
3,887,5252,0,1,11,0.557,0.979,6,-4.254,0.0489,...,4.81,0.000000,0.375,0.285107,1,165.0,16.0,83.0,6.27,0
4,953,5066,33,5,0,0.587,0.977,10,-3.589,0.0711,...,4.46,0.000000,0.000,0.292775,1,165.0,16.0,83.0,6.27,0


## Retraining after feature selection

In [49]:
df_2 = model_df.copy()
df_2.drop([col for col, importance in sorted_feature_importance if importance < 90], axis=1, inplace=True)
X_train, X_test, y_train, y_test = dividing_df(df_2)
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)


In [50]:
gbm = lgb.train(params, train_data, num_boost_round=100, valid_sets=[test_data])
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
y_pred_class = [1 if prob >= 0.5 else 0 for prob in y_pred]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000341 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3456
[LightGBM] [Info] Number of data points in the train set: 2000, number of used features: 23
[LightGBM] [Info] Start training from score 0.498000


In [51]:
mse = mean_squared_error(y_test, y_pred)
print(f"The Mean Squared Error (MSE) on test set: {mse}")
# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred_class)
print(f"The accuracy of the model is: {accuracy}")

The Mean Squared Error (MSE) on test set: 0.15205507539970337
The accuracy of the model is: 0.768


In [52]:
df_2.head()

,track_id_encoded,track_popularity,playlist_subgenre,danceability,energy,loudness,speechiness,acousticness,liveness,valence,tempo,duration_minutes,affinity_score,avg_bpm,age,avg_cadence,avg_pace,likes_song
0,215,5,18,0.675,0.868,-6.360,0.0392,0.00107,0.3740,0.0376,128.007,5.07,0.287000,165.0,16.0,83.0,6.27,0
1,2080,62,9,0.605,0.584,-7.043,0.1970,0.28600,0.2350,0.4640,125.802,4.09,0.504277,165.0,16.0,83.0,6.27,1
2,521,68,7,0.673,0.814,-5.411,0.0367,0.01330,0.3430,0.7280,147.193,4.20,0.395855,165.0,16.0,83.0,6.27,0
3,887,0,11,0.557,0.979,-4.254,0.0489,0.00130,0.0483,0.8560,127.998,4.81,0.285107,165.0,16.0,83.0,6.27,0
4,953,33,0,0.587,0.977,-3.589,0.0711,0.03530,0.5360,0.5190,146.518,4.46,0.292775,165.0,16.0,83.0,6.27,0


# Generating real data: 
Per user, the input will be: 
- avg_bpm	
- age
- avg_cadence
- avg_pace



In [53]:
#  MOCK VALUES !!!
avg_bpm = 180
age = 22
avg_cadence = 160
avg_pace = 6.5

user_traits = {'avg_bpm':avg_bpm, 'age': age, 'avg_cadence': avg_cadence, 'avg_pace': avg_pace}

In [54]:
df_2 = df_2[['avg_bpm', 'age', 'avg_cadence', 'avg_pace'] + [col for col in df_2.columns if col not in ['avg_bpm', 'age', 'avg_cadence', 'avg_pace']]]
df_2.drop(['likes_song', 'affinity_score'], axis=1, inplace=True)

### Imputting the user's values into the song df

In [55]:
input_columns = ['avg_bpm', 'age', 'avg_cadence', 'avg_pace']
for col in input_columns:
    df_2[col] = user_traits[col]
    
df_2.head()

,avg_bpm,age,avg_cadence,avg_pace,track_id_encoded,track_popularity,playlist_subgenre,danceability,energy,loudness,speechiness,acousticness,liveness,valence,tempo,duration_minutes
0,180,22,160,6.5,215,5,18,0.675,0.868,-6.360,0.0392,0.00107,0.3740,0.0376,128.007,5.07
1,180,22,160,6.5,2080,62,9,0.605,0.584,-7.043,0.1970,0.28600,0.2350,0.4640,125.802,4.09
2,180,22,160,6.5,521,68,7,0.673,0.814,-5.411,0.0367,0.01330,0.3430,0.7280,147.193,4.20
3,180,22,160,6.5,887,0,11,0.557,0.979,-4.254,0.0489,0.00130,0.0483,0.8560,127.998,4.81
4,180,22,160,6.5,953,33,0,0.587,0.977,-3.589,0.0711,0.03530,0.5360,0.5190,146.518,4.46


### Predicting for that specific user

In [56]:
y_pred = gbm.predict(df_2, num_iteration=gbm.best_iteration, predict_disable_shape_check=True)
y_pred_class = [1 if prob >= 0.5 else 0 for prob in y_pred]

In [57]:
df_2['y_pred'] = y_pred_class

In [58]:
df_track_artist_mapping = pd.read_csv('../data_prep/track_artist_mapping.csv')

# Create a mapping dictionary from the DataFrame
artist_mapping = pd.Series(df_track_artist_mapping.track_artist.values, index=df_track_artist_mapping.label).to_dict()

# Use the mapping to replace the encoded labels in 'track_artist'
df['track_artist'] = df['track_artist'].map(artist_mapping)

In [59]:
df.head()

,track_id_encoded,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,intensity_zone,intensity_zone_encoded,cluster,affinity_score,gender,avg_bpm,age,avg_cadence,avg_pace,likes_song
0,215,0hv6ku4431kPrjbpBBspw3,Animals - Original Mix,Martin Garrix,5,0N5QcojgdA6aP9oQ7yYG9A,1148,2013-07-01,gym melbourne house,5BqOZpVEqRDfZScvW1QUyA,...,Zone 4 (Hard),3,10,0.287000,1,165.0,16.0,83.0,6.27,0
1,2080,6uYAPqxP69zfpCbsTHXgiz,Cater 2 U,Destiny's Child,62,0b6ivSFfDs38MG7aLn9rvO,4221,2004-11-16,hip hop rnb,0Ar0Ng9DlAWZtSPBvOQgOa,...,Zone 3 (Moderate),2,14,0.504277,1,165.0,16.0,83.0,6.27,1
2,521,1gzIbdFnGJ226LTl0Cn2SX,White Wedding - Pt. 1,Billy Idol,68,5ebh0MxkqELmk50xHj2b4k,2079,1982-01-01,hard rock cafe,55ybnG2z8rpTIerRfnCsw6,...,Zone 5 (Maximum),4,0,0.395855,1,165.0,16.0,83.0,6.27,0
3,887,2vX03yHcVtFUg2ZMuzhLhS,Outlaw - Original Mix,La Fuente,0,7jV5akfXEXsoNsqrekDCyX,12116,2014-04-21,latin village,3JcJCsvBnRsA6oc7eUYDSB,...,Zone 4 (Hard),3,10,0.285107,1,165.0,16.0,83.0,6.27,0
4,953,38LrhW3eQSt3cK88dkvl5e,Misty Night Cruising,Kiyotaka Sugiyama,33,24sd1ZTrjjabj0A54XK0RP,11054,1984-12-21,city pop シティーポップ,3j2osvmecEao5nmo9jZ5df,...,Zone 5 (Maximum),4,0,0.292775,1,165.0,16.0,83.0,6.27,0


### Retrieving the song name and the pred value

In [60]:
merged_df = df.merge(df_2[['track_id_encoded', 'y_pred']], how='left', on='track_id_encoded', suffixes=('', '_y'))
merged_df.drop(['likes_song', 'affinity_score', 'gender'], axis=1, inplace=True)
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]
[['track_name', 'y_pred','track_name' ]]



[['track_name', 'y_pred', 'track_name']]

In [61]:

# Get the track names and track artists of the songs that the model predicts the user will like
track_names_artists = merged_df.loc[merged_df['y_pred'] == 1]

track_names_artists.head()

num_samples = min(len(track_names_artists), 10)
random_subset = track_names_artists.sample(n=num_samples)

random_subset


,track_id_encoded,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,playlist_name_sentiment,track_name_sentiment,intensity_zone,intensity_zone_encoded,cluster,avg_bpm,age,avg_cadence,avg_pace,y_pred
2622,2189,7GBO6BJdevV7XojDlP7aE8,Thank You Lord (For Being There For Me),Fred Hammond,31,2mI4JRPhTGEZXw3g6dBCtA,12996,2000-04-18,gospel,0pItJZBDZFvgAOdFnSITnJ,...,0.000000,0.0000,Zone 2 (Light),1,16,142.0,38.0,50.0,6.06,1
1387,856,2nluoJKvE7rTBN7kve8dnr,Spill The Wine,Eric Burdon,51,6HeXDX3B46Utsd0hL7nBUG,16124,1976-01-01,classic rock retrogamer,6gUFdcGzKAHyDXY9TKC6cP,...,0.166667,0.0000,Zone 3 (Moderate),2,4,79.0,13.0,118.0,5.02,1
1917,911,30ZhaZWc3F9gvOdA3GFYn1,One More Time,Alberto Ciccarini,34,3qGXwdNRKLkhZ213zEtvzg,11926,2019-06-21,tropical house,06zrBJ5cts5aemZmqe80J7,...,0.000000,0.5000,Zone 2 (Light),1,2,216.0,48.0,64.0,5.81,1
2697,76,0Fc9QSMX6fgEBgDjUax7t7,Ruin My Life - Steve James Remix,Zara Larsson,58,6rWRg6yLGcLdG0IRaCB9LN,13774,2018-11-23,pop remix,37i9dQZF1DXcZDD7cfEKhW,...,0.000000,0.0000,Zone 3 (Moderate),2,12,236.0,33.0,116.0,6.35,1
791,336,17dzcc3DPZOoW4mtq1r1Dr,Blackbird,Jonas Aden,3,1nPjnw2l59kVVizsh9hwHf,2153,2019-02-01,parookaville big room,6vbHQdtfmexb0ptmBSpemm,...,0.000000,0.0000,Zone 3 (Moderate),2,10,107.0,45.0,60.0,6.64,1
2146,2006,6hIzBgQvJDG1qp6WHU7gRI,The Line,Audax,44,59a3l846nshXyZ2pu3qGan,16543,2019-11-06,techno house best collection top dj electronic...,3QEYvCsVXZj8KuzE0bDmcI,...,0.500000,0.0000,Zone 3 (Moderate),2,5,143.0,30.0,60.0,5.36,1
1253,1660,5Tp4UJvnsF4Zd05k0zXUte,CPR,Summer Walker,68,4hTM8jZ8DCCqglmZUDMSPz,9066,2018-10-19,neo soul,44d7ppo4cggZJmzH2WOhAc,...,0.000000,0.0000,Zone 5 (Maximum),4,14,86.0,25.0,38.0,6.85,1
2167,74,0FE8gz0tOBbAnOp90WbzsK,Psychic Driving,Soft Metals,17,1R5JJa9OEfnXivtuusDLjr,14767,2011-01-01,electropop play,7p30DzTAgW6OhspSXHTI88,...,0.000000,0.0000,Zone 3 (Moderate),2,10,143.0,30.0,60.0,5.36,1
714,612,21LMl54FIe7J5oOoZzgeiQ,Ang Huling El Bimbo,Eraserheads,61,2DIpgoeZAbwk3wF1qSDj8v,1189,2004-01-26,pinoy classic rock,37i9dQZF1DWYDQ8wBxd7xt,...,0.166667,0.0000,Zone 1 (Very Light),0,4,127.0,29.0,117.0,6.53,1
901,777,2W0cQltLhmrTt1Ild75DuV,Have Yourself a Merry Little Christmas,Luther Vandross,28,5pyHOj2s15KUZBsBUe9OmQ,10441,2012-09-28,christmas soul,6FZYc2BvF7tColxO8PBShV,...,0.000000,-0.1875,Zone 2 (Light),1,1,245.0,14.0,8.0,7.00,1


### To get real data: each user has to corroborar que el prediction era real, y si no hay que cambiarlo. 

In [62]:
import requests

def read_access_token(file_path):
    with open(file_path, 'r') as file:
        return file.read().strip()
    
access_token = read_access_token('access_token.txt')


def get_spotify_user_profile(access_token):
    url = 'https://api.spotify.com/v1/me'
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()  # Returns user profile data
    else:
        print(f"Error fetching user profile: {response.status_code}")
        print(f"Response: {response.json()}")
        return None

user_profile = get_spotify_user_profile(access_token)
print(user_profile)

def get_spotify_top_data(access_token, data_type='tracks'):
    url = f'https://api.spotify.com/v1/me/top/{data_type}'
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return [item['id'] for item in response.json()['items']]
    else:
        # Log the status code and response for debugging
        print(f"Error fetching top {data_type}: {response.status_code}")
        print(f"Response: {response.json()}")
        return []


access_token = read_access_token('access_token.txt')
top_tracks = get_spotify_top_data(access_token, 'tracks')
top_artists = get_spotify_top_data(access_token, 'artists')

{'display_name': 'santiagofischel', 'external_urls': {'spotify': 'https://open.spotify.com/user/santiagofischel'}, 'href': 'https://api.spotify.com/v1/users/santiagofischel', 'id': 'santiagofischel', 'images': [{'url': 'https://i.scdn.co/image/ab67757000003b82767ddaf83ec34926931494c7', 'height': 64, 'width': 64}, {'url': 'https://i.scdn.co/image/ab6775700000ee85767ddaf83ec34926931494c7', 'height': 300, 'width': 300}], 'type': 'user', 'uri': 'spotify:user:santiagofischel', 'followers': {'href': None, 'total': 68}}


In [63]:
top_tracks

['5kDbL56GBwZwGSOxuFODSX',
 '2DsUXoqK3HXx2Sat09W7uN',
 '2VEZx7NWsZ1D0eJ4uv5Fym',
 '0IedgQjjJ8Ad4B3UDQ5Lyn',
 '24yrJJkmVUMkJHvCaR85BN',
 '4jqu753tYcQ1MsoH9wXDSW',
 '3qebSLgUwaEqbwCM5sj5Kh',
 '0L7UzdYmOkLhHoDLEcUa8f',
 '6cfVDaIdvDtYH91RqC6Wox',
 '0o73cSWmV0YJeHvzDbJDdn',
 '2f86Ae3R4OKAu1lEIqouih',
 '1Y1jtaZOJYgXBl7lKdLc9o',
 '40EXFgNMOIQRQOwJZ8mJkC',
 '6prWQW8NVawRNIeULPimTz',
 '16gJmoOqRt2lKmlXtDpWgH',
 '2CHmgtK8OCL28WtIK96u4N',
 '6cvMmviJTZTBOFh2ohwZg6',
 '03L2AoiRbWhvt7BDMx1jUB',
 '0YuzpR1mrMXQe20hdijnZv',
 '7wdzLe2Gsx1RGqbvYZHASz']

In [64]:
top_artists

['3TVXtAsR1Inumwj472S9r4',
 '2LRoIwlKmHjgvigdNGBHNo',
 '5K4W6rqBFWDnAN6FQUkS6x',
 '4q3ewBCX7sLwd24euuV69X',
 '3nFkdlSjzX9mRTtwJOzDYB',
 '6t38N9HASTn9ca0PIxfReQ',
 '2YZyLoL8N0Wb9xBt1NhZWg',
 '2pAWfrd7WFF3XhVt9GooDL',
 '7JXB79cwLMZDYTwbDe7lfb',
 '0hEurMDQu99nJRq8pTxO14',
 '2h93pZq0e7k5yf4dywlkpM',
 '1Xyo4u8uXC1ZmMpatF05PJ',
 '4tZwfgrHOc3mvqYlEYSvVi',
 '0wn2qDKzeFlhjRUtJAwJjp',
 '20qISvAhX20dpIbOOzGK3q',
 '7tYKF4w9nC0nq9CsPZTHyP',
 '5XJDexmWFLWOkjOEjOVX3e',
 '3DplAKAVidNrhpivWNLMWC',
 '5J7rXWjtn5HzUkJ4Jet8Fr',
 '3oKRxpszQKUjjaHz388fVA']

In [65]:
merged_df.head()

,track_id_encoded,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,playlist_name_sentiment,track_name_sentiment,intensity_zone,intensity_zone_encoded,cluster,avg_bpm,age,avg_cadence,avg_pace,y_pred
0,215,0hv6ku4431kPrjbpBBspw3,Animals - Original Mix,Martin Garrix,5,0N5QcojgdA6aP9oQ7yYG9A,1148,2013-07-01,gym melbourne house,5BqOZpVEqRDfZScvW1QUyA,...,0.000000,0.375,Zone 4 (Hard),3,10,165.0,16.0,83.0,6.27,0
1,2080,6uYAPqxP69zfpCbsTHXgiz,Cater 2 U,Destiny's Child,62,0b6ivSFfDs38MG7aLn9rvO,4221,2004-11-16,hip hop rnb,0Ar0Ng9DlAWZtSPBvOQgOa,...,0.000000,0.000,Zone 3 (Moderate),2,14,165.0,16.0,83.0,6.27,1
2,521,1gzIbdFnGJ226LTl0Cn2SX,White Wedding - Pt. 1,Billy Idol,68,5ebh0MxkqELmk50xHj2b4k,2079,1982-01-01,hard rock cafe,55ybnG2z8rpTIerRfnCsw6,...,-0.291667,0.000,Zone 5 (Maximum),4,0,165.0,16.0,83.0,6.27,0
3,887,2vX03yHcVtFUg2ZMuzhLhS,Outlaw - Original Mix,La Fuente,0,7jV5akfXEXsoNsqrekDCyX,12116,2014-04-21,latin village,3JcJCsvBnRsA6oc7eUYDSB,...,0.000000,0.375,Zone 4 (Hard),3,10,165.0,16.0,83.0,6.27,0
4,953,38LrhW3eQSt3cK88dkvl5e,Misty Night Cruising,Kiyotaka Sugiyama,33,24sd1ZTrjjabj0A54XK0RP,11054,1984-12-21,city pop シティーポップ,3j2osvmecEao5nmo9jZ5df,...,0.000000,0.000,Zone 5 (Maximum),4,0,165.0,16.0,83.0,6.27,0


In [66]:
# Step 1: Filter merged_df to get only recommended songs
recommended_songs_df = merged_df[merged_df['y_pred'] == 1]

# Step 2: Extract track IDs from the recommended songs
# Assuming 'track_id' column contains the Spotify track IDs
recommended_track_ids = recommended_songs_df['track_id'].tolist()

# Step 3: Compare with your top tracks from Spotify
# Assuming 'top_tracks' is a list of your top track IDs from Spotify
liked_songs_df = recommended_songs_df[recommended_songs_df['track_id'].isin(top_tracks)]

# Step 4: Optionally, review the DataFrame of matched songs
print(liked_songs_df.head())

Empty DataFrame
Columns: [track_id_encoded, track_id, track_name, track_artist, track_popularity, track_album_id, track_album_name, track_album_release_date, playlist_name, playlist_id, playlist_genre, playlist_subgenre, danceability, energy, key, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo, duration_minutes, track_name_preprocessed, playlist_name_sentiment, track_name_sentiment, intensity_zone, intensity_zone_encoded, cluster, avg_bpm, age, avg_cadence, avg_pace, y_pred]
Index: []

[0 rows x 34 columns]


In [67]:
recommended_songs_df.head()

,track_id_encoded,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,playlist_name_sentiment,track_name_sentiment,intensity_zone,intensity_zone_encoded,cluster,avg_bpm,age,avg_cadence,avg_pace,y_pred
1,2080,6uYAPqxP69zfpCbsTHXgiz,Cater 2 U,Destiny's Child,62,0b6ivSFfDs38MG7aLn9rvO,4221,2004-11-16,hip hop rnb,0Ar0Ng9DlAWZtSPBvOQgOa,...,0.0,0.0,Zone 3 (Moderate),2,14,165.0,16.0,83.0,6.27,1
5,1690,5Yp8d1wlkpup010VvKFEP2,Catch & Release - Deepend Remix Extended Version,Matt Simons,3,6nXdrQwZA9LnTLxIAD8smC,2990,2015-09-18,tropical morning,0vbtbmyihyvl2MVYrdKdGw,...,0.0,0.0,Zone 2 (Light),1,14,165.0,16.0,83.0,6.27,1
7,1999,6ftZKIYKzEZBBGCKK1SCGX,Whoopedoo - Varrick Frost Remix,Balduin,21,3QAHeoozh7V7vEkALI0iZT,18802,2019-07-26,electro swing top,5M7xLCQTYJL0lbKIrUn3a4,...,0.5,0.0,Zone 4 (Hard),3,10,165.0,16.0,83.0,6.27,1
10,525,1hf823Dtg12gHo3d2R6Ne8,Nonno Hollywood,Enrico Nigiotti,50,1Ehq0w6zLcZAM0CzcKocI6,11560,2019-02-06,musica italiana playlist pop canzoni italiane,6kVFIQBhLT4003iw2WWEv1,...,0.0,0.0,Zone 3 (Moderate),2,14,165.0,16.0,83.0,6.27,1
11,525,1hf823Dtg12gHo3d2R6Ne8,Nonno Hollywood,Enrico Nigiotti,50,1Ehq0w6zLcZAM0CzcKocI6,11560,2019-02-06,musica italiana playlist pop canzoni italiane,6kVFIQBhLT4003iw2WWEv1,...,0.0,0.0,Zone 3 (Moderate),2,14,165.0,16.0,83.0,6.27,1


In [68]:
def get_playlist_tracks(access_token, playlist_id):
    url = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks'
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers)
    return [item['track']['id'] for item in response.json()['items']] if response.status_code == 200 else []

# Get all playlists
def get_all_user_playlists(access_token):
    url = 'https://api.spotify.com/v1/me/playlists'
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers)
    playlists = response.json()['items'] if response.status_code == 200 else []
    return playlists

# Fetch all tracks from all playlists
def get_all_tracks_from_playlists(access_token):
    playlists = get_all_user_playlists(access_token)
    all_tracks = set()
    for playlist in playlists:
        tracks = get_playlist_tracks(access_token, playlist['id'])
        all_tracks.update(tracks)
    return all_tracks

access_token = read_access_token('access_token.txt')  # Update the path to your access token file
playlist_tracks = get_all_tracks_from_playlists(access_token)


In [69]:
playlist_tracks

{'00NAQYOP4AmWR549nnYJZu',
 '00hdjyXt6MohKnCyDmhxOL',
 '01DOm09B2AbiTARAbXVI6g',
 '03t183ijgT3kua3Bk7XHpP',
 '06s3QtMJVXw1AJX3UfvZG1',
 '07w6jp4Fq8GGqhLoDYoVkb',
 '0AjNBgkkdeUyj4HucyGRr8',
 '0DMRKqrpd0QO4RWiEdMib0',
 '0ESJlaM8CE1jRWaNtwSNj8',
 '0G4QCpJMA6u8MGXxzg4gm9',
 '0GONea6G2XdnHWjNZd6zt3',
 '0GbrwlmnRMJOQ5bRV9QcZa',
 '0HB4odbGJbgR0ffA4rJoxS',
 '0HDudjyoG3UpWIMAJrXIaB',
 '0I3q5fE6wg7LIfHGngUTnV',
 '0L7UzdYmOkLhHoDLEcUa8f',
 '0Nwar8rweBUyfyuoyjjUvk',
 '0OIkGLJOrJtrQ9pm805I5m',
 '0P6AWOA4LG1XOctzaVu5tt',
 '0RbW8kWozrVSIGb4V13o0o',
 '0S9ZVNTNRzLMgb96tmmG1u',
 '0SbDNXZYqfsMarINcb72X5',
 '0SqqAgdovOE24BzxIClpjw',
 '0V4l4GQhgnWQGtCWpvA7va',
 '0W1jh3ExLv0nMvvmNJOBxh',
 '0Wi7iEDGFY7I7NkYU6gS4n',
 '0aVrpFRLlrd5zVyPXWP3mS',
 '0aym2LBJBk9DAYuHHutrIl',
 '0bRXwKfigvpKZUurwqAlEh',
 '0c64nOsaJu7LSrX9pI4124',
 '0dFdGPVLs3k0z9ezYWZzUa',
 '0dWOFwdXrbBUYqD9DLsoyK',
 '0dbTQYW3Ad1FTzIA9t90E8',
 '0eXIlXgGQTD6hezaxdl3wO',
 '0gzqZ9d1jIKo9psEIthwXe',
 '0h4jHdNzUrV9eGR2PT5R8c',
 '0jeS8PyyIRAG0A3t5DtL71',
 

In [70]:

# Assuming recommended_songs_df and playlist_tracks are defined
matches = recommended_songs_df[recommended_songs_df['track_id'].isin(playlist_tracks)]

In [71]:
# Append matches to df_liked_songs
df_liked_songs = df_liked_songs._append(matches, ignore_index=True)

# Save the updated DataFrame
df_liked_songs.to_csv('liked_songs.csv', index=False)

In [72]:
matches.head()

,track_id_encoded,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,playlist_name_sentiment,track_name_sentiment,intensity_zone,intensity_zone_encoded,cluster,avg_bpm,age,avg_cadence,avg_pace,y_pred
245,238,0mfCH4nfpshBQR9ueZP5eP,Girls Need Love (with Drake) - Remix,Summer Walker,10,3i9w4RgUuxBxUVaarH91kx,6351,2019-02-10,contemporary urban,6wyJ4bsjZaUKa9f6GeZlAO,...,0.083333,0.5,Zone 2 (Light),1,1,108.0,44.0,101.0,6.02,1
246,238,0mfCH4nfpshBQR9ueZP5eP,Girls Need Love (with Drake) - Remix,Summer Walker,10,3i9w4RgUuxBxUVaarH91kx,6351,2019-02-10,contemporary urban,6wyJ4bsjZaUKa9f6GeZlAO,...,0.083333,0.5,Zone 2 (Light),1,1,108.0,44.0,101.0,6.02,1
495,91,0I3q5fE6wg7LIfHGngUTnV,Ms. Jackson,OutKast,77,2tm3Ht61kqqRZtIYsBjxEj,15108,2000-10-31,saxophone hip hop r b lounge jazz rap,2ujrP2rPxu8ao7wNeG1ZOY,...,0.000000,0.0,Zone 1 (Very Light),0,9,181.0,19.0,111.0,5.71,1
864,713,2JvzF1RMd7lE3KmFlsyZD8,MIDDLE CHILD,J. Cole,86,3XzSOIE6zGLliuqsVGLmUc,9962,2019-01-23,trap americana,7tkgK1tm9hYkWp7EFyOcAr,...,0.000000,0.0,Zone 3 (Moderate),2,9,101.0,53.0,70.0,7.18,1
978,238,0mfCH4nfpshBQR9ueZP5eP,Girls Need Love (with Drake) - Remix,Summer Walker,10,3i9w4RgUuxBxUVaarH91kx,6351,2019-02-10,contemporary urban,6wyJ4bsjZaUKa9f6GeZlAO,...,0.083333,0.5,Zone 2 (Light),1,1,73.0,46.0,119.0,7.27,1


In [73]:
df_liked_songs.head()

,track_id_encoded,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,playlist_name_sentiment,track_name_sentiment,intensity_zone,intensity_zone_encoded,cluster,avg_bpm,age,avg_cadence,avg_pace,y_pred
0,2207.0,7KA4W4McWYRpgf0fWsJZWB,See You Again (feat. Kali Uchis),"Tyler, The Creator",79.0,2nkto6YNI4rUYTLqEwWJ3o,5800.0,2017-07-21,indie poptimism,1E5VdGGWkpDQ682gdvJAZH,...,0.000000,0.0,Zone 1 (Very Light),0.0,1.0,127.0,20.0,9.0,6.71,1.0
1,91.0,0I3q5fE6wg7LIfHGngUTnV,Ms. Jackson,OutKast,77.0,2tm3Ht61kqqRZtIYsBjxEj,15108.0,2000-10-31,saxophone hip hop r b lounge jazz rap,2ujrP2rPxu8ao7wNeG1ZOY,...,0.000000,0.0,Zone 1 (Very Light),0.0,9.0,181.0,19.0,111.0,5.71,1.0
2,1444.0,4j3GWI86JvSaF0BLdmgcfF,Summertime Magic,Childish Gambino,70.0,15k99o4mZJ9mfpQGIOrZ45,15411.0,2018-07-11,project contemporary,6HaCi9bqaiuSZEDfCEmwyo,...,0.166667,0.5,Zone 2 (Light),1.0,2.0,239.0,27.0,66.0,5.29,1.0
3,759.0,2TrzgfBaSP387EEFDgG4FW,Verano Azul,Juan Magán,1.0,6GxHVUwxvviiCffH7coRPC,18029.0,2009-01-01,mix latin,6IS6XTdbS9qJZgfjNKgpB8,...,0.000000,0.0,Zone 3 (Moderate),2.0,15.0,242.0,58.0,84.0,7.85,1.0
4,2037.0,6mAN61JH0dzyZpWslS11jy,Fantasias,Rauw Alejandro,92.0,1Flcx9eDuv7pTGM9nJBmGL,5483.0,2019-08-29,latin pop pop latino actual,2kKzN3kRYDzBctlaWs7CP2,...,0.000000,0.0,Zone 1 (Very Light),0.0,12.0,236.0,33.0,116.0,6.35,1.0


In [74]:
def get_user_feedback(recommended_songs_df):
    liked_songs = []
    for _, song in recommended_songs_df.iterrows():
        # Assume a simple Yes/No prompt for the user to indicate if they like the song
        feedback = input(f"Do you like this song? {song['track_name']} by {song['track_artist']} (Yes/No): ").lower()
        if feedback == 'yes':
            liked_songs.append(song)
    
    # Convert the list of liked songs to a DataFrame
    df_liked_songs = pd.DataFrame(liked_songs)
    
    return df_liked_songs

# Use the function with the random_subset DataFrame
new_liked_songs = get_user_feedback(random_subset)

In [75]:
# Append new liked songs from feedback to df_liked_songs
df_liked_songs = df_liked_songs._append(new_liked_songs, ignore_index=True)

# Save the updated DataFrame
df_liked_songs.to_csv('liked_songs.csv', index=False)

In [76]:
df_liked_songs_final = pd.read_csv('liked_songs.csv')   

In [77]:
df_liked_songs_final

,track_id_encoded,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,playlist_name_sentiment,track_name_sentiment,intensity_zone,intensity_zone_encoded,cluster,avg_bpm,age,avg_cadence,avg_pace,y_pred
0,2207.0,7KA4W4McWYRpgf0fWsJZWB,See You Again (feat. Kali Uchis),"Tyler, The Creator",79.0,2nkto6YNI4rUYTLqEwWJ3o,5800.0,2017-07-21,indie poptimism,1E5VdGGWkpDQ682gdvJAZH,...,0.000000,0.000000,Zone 1 (Very Light),0.0,1.0,127.0,20.0,9.0,6.71,1.0
1,91.0,0I3q5fE6wg7LIfHGngUTnV,Ms. Jackson,OutKast,77.0,2tm3Ht61kqqRZtIYsBjxEj,15108.0,2000-10-31,saxophone hip hop r b lounge jazz rap,2ujrP2rPxu8ao7wNeG1ZOY,...,0.000000,0.000000,Zone 1 (Very Light),0.0,9.0,181.0,19.0,111.0,5.71,1.0
2,1444.0,4j3GWI86JvSaF0BLdmgcfF,Summertime Magic,Childish Gambino,70.0,15k99o4mZJ9mfpQGIOrZ45,15411.0,2018-07-11,project contemporary,6HaCi9bqaiuSZEDfCEmwyo,...,0.166667,0.500000,Zone 2 (Light),1.0,2.0,239.0,27.0,66.0,5.29,1.0
3,759.0,2TrzgfBaSP387EEFDgG4FW,Verano Azul,Juan Magán,1.0,6GxHVUwxvviiCffH7coRPC,18029.0,2009-01-01,mix latin,6IS6XTdbS9qJZgfjNKgpB8,...,0.000000,0.000000,Zone 3 (Moderate),2.0,15.0,242.0,58.0,84.0,7.85,1.0
4,2037.0,6mAN61JH0dzyZpWslS11jy,Fantasias,Rauw Alejandro,92.0,1Flcx9eDuv7pTGM9nJBmGL,5483.0,2019-08-29,latin pop pop latino actual,2kKzN3kRYDzBctlaWs7CP2,...,0.000000,0.000000,Zone 1 (Very Light),0.0,12.0,236.0,33.0,116.0,6.35,1.0
5,817.0,2eU63GLRBeMOszGRTCEE9E,California Love - Original Version,2Pac,2.0,6KwC7GaK6R7Uphby8bnmqk,242.0,1998-01-01,oldschool gangsta rap,0t9TeyXE2mWe70JT3YwSoH,...,0.000000,0.437500,Zone 1 (Very Light),0.0,15.0,202.0,29.0,97.0,5.44,1.0
6,1113.0,3bcYb4HwWlytUFLny0e0Q1,With Or Without You,U2,2.0,586ZRfgsIckfcKvHVcGM4V,16494.0,1987-03-10,pop rock hits album tracks,0XOIK4m26aeYSD61E5nSVW,...,0.000000,0.000000,Zone 2 (Light),1.0,4.0,192.0,19.0,68.0,5.86,1.0
7,73.0,0FDzzruyVECATHXKHFs9eJ,A Sky Full of Stars,Coldplay,78.0,2G4AUqfwxcV1UdQjm2ouYr,6315.0,2014-05-19,NaN,44p8nNLe4fGfUeArS3MaIX,...,0.000000,0.350000,Zone 3 (Moderate),2.0,2.0,91.0,24.0,101.0,7.67,1.0
8,1867.0,6BnONjR7itGMEqwxKTIlRM,Safe And Sound,Capital Cities,13.0,5ps3FwS0qGdRCvXop8q9vn,8004.0,2013-01-01,electropop,0cuHKz65ZPqBX1brG8djlg,...,0.000000,0.450000,Zone 3 (Moderate),2.0,2.0,230.0,54.0,100.0,5.74,1.0
9,249.0,0oWHLtxWeMJhmwxtrxhNK0,Welcome To The Jungle,Guns N' Roses,1.0,1oR3KrPIp4CbagPa3PhtPp,1231.0,1987-01-01,classic rock rock classics rock rock rock rock...,3Ho3iO0iJykgEQNbjB2sic,...,0.166667,0.800000,Zone 3 (Moderate),2.0,2.0,81.0,57.0,95.0,7.08,1.0
